In [6]:
import pandas as pd
import numpy as np # Importamos numpy para usar 'NaN' na limpeza de notas

# Definição das colunas a serem carregadas

colunas_definitivas = [
    # Notas
    'NU_NOTA_CN', 
    'NU_NOTA_CH', 
    'NU_NOTA_LC', 
    'NU_NOTA_MT', 
    'NU_NOTA_REDACAO',
    # Informações da Escola
    'TP_DEPENDENCIA_ADM_ESC',
    'SG_UF_ESC',
    'TP_LOCALIZACAO_ESC',
    # Status da Redação para limpeza
    'TP_STATUS_REDACAO'
]

caminho_resultados = 'DADOS/RESULTADOS_2024.csv'

try:
    # Usando o parâmetro 'usecols' com a lista de colunas
    df = pd.read_csv(
        caminho_resultados,
        sep=';',
        encoding='latin-1',
        usecols=colunas_definitivas
    )
    print("Dados carregados com sucesso!")
except Exception as e:
    print(f"Ocorreu um erro ao ler o arquivo: {e}")

# Filtragem e Limpeza 
print("\nIniciando filtragem e limpeza dos dados...")

# Filtro: apenas escolas de São Paulo
df_sp = df[df['SG_UF_ESC'] == 'SP'].copy()
print(f"-> {len(df_sp)} registros encontrados para escolas em São Paulo.")

# Garantia de que estamos analisando apenas redações válidas (status 1 = 'Sem problemas')
df_sp = df_sp[df_sp['TP_STATUS_REDACAO'] == 1]
print(f"-> {len(df_sp)} registros restantes após filtrar por redações válidas.")

# Mapeamento e Criação de Categorias
print("\nCriando categorias para 'Tipo de Escola' e 'Localização'")

# Criando uma nova coluna com a classificação do Tipo de Escola (Pública | Privada), usando o método apply do pandas
# 1, 2, 3 -> Pública | 4 -> Privada
df_sp['TIPO_ESCOLA'] = df_sp['TP_DEPENDENCIA_ADM_ESC'].apply(lambda x: 'Pública' if x in [1, 2, 3] else 'Privada')

# Criando uma nova coluna com a classificação da localização para 'Urbana' ou 'Rural'
# 1 -> Urbana | 2 -> Rural
df_sp['LOCALIZACAO'] = df_sp['TP_LOCALIZACAO_ESC'].apply(lambda x: 'Urbana' if x == 1 else 'Rural')

# Análise e Exibição dos Resultados ---
print("\nCalculando médias de desempenho")

# Agrupamento por TIPO_ESCOLA e calculo da média das notas
media_por_tipo = df_sp.groupby('TIPO_ESCOLA')[['NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT','NU_NOTA_REDACAO']].mean().round(2)

print("\nMÉDIA DE NOTAS POR TIPO DE ESCOLA (SP)")
print(media_por_tipo)

# Agrupamento por TIPO_ESCOLA e LOCALIZACAO
media_por_localizacao = df_sp.groupby(['TIPO_ESCOLA', 'LOCALIZACAO'])[['NU_NOTA_CN','NU_NOTA_CH','NU_NOTA_LC','NU_NOTA_MT','NU_NOTA_REDACAO']].mean().round(2)

print("\n\nMÉDIA POR TIPO E LOCALIZAÇÃO DA ESCOLA (SP)")
print(media_por_localizacao)

Dados carregados com sucesso!

Iniciando filtragem e limpeza dos dados...
-> 285199 registros encontrados para escolas em São Paulo.
-> 220464 registros restantes após filtrar por redações válidas.

Criando categorias para 'Tipo de Escola' e 'Localização'

Calculando médias de desempenho

MÉDIA DE NOTAS POR TIPO DE ESCOLA (SP)
             NU_NOTA_CN  NU_NOTA_CH  NU_NOTA_LC  NU_NOTA_MT  NU_NOTA_REDACAO
TIPO_ESCOLA                                                                 
Privada          552.70      569.85      568.37      620.84           747.68
Pública          483.15      508.00      528.76      515.42           620.20


MÉDIA POR TIPO E LOCALIZAÇÃO DA ESCOLA (SP)
                         NU_NOTA_CN  NU_NOTA_CH  NU_NOTA_LC  NU_NOTA_MT  \
TIPO_ESCOLA LOCALIZACAO                                                   
Privada     Rural            536.53      543.91      555.56      586.11   
            Urbana           552.78      569.97      568.43      621.00   
Pública     Rural